# Supporting Notebook in CS1 2024

In this Notebook, we will look at a few simple systems, and qualitatively assess how different control strategies work on each one. 

## First-order model
Let us consider the system below, which roughly models the speed dynamics of an old-fashioned car, without cruise control or other modern speed control devices. 

![Image Title](./media/sys1.png)


Notice that there are two **signals** (i.e., functions of time) going into the "car" **system** (i.e., something that converts one or more input signals into one or more output signals):
- One of the input signals represents how much the gas (or brake) pedal is pressed, as a function of time. This is called the **control** signal, since this is how we try to influence the speed of the car. (This is an example of an *endogeneous* signal, i.e., something that is controlled by the user)
- The other represents the slope of the road, with a positive number referring to an uphill slope. This is a **disturbance** signal, and is typically something generated outside of the system (i.e., an *exogeneous* signal), beyond the user's knowledge or control. 
- The system in turn generates an **output** signal, which in this case represents how the speed of the vehicle changes over time. 

Assume that it is desired to maintain a speed of $50$ km/h. This is called the **reference** signal, and for now let us set it to a constant. Note that this signal does not appear in our system yet, this is just a convenience for us to keep in mind what is the control objective. 

In [118]:
import control as ct
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, Image

# Define a time vector
time = np.linspace(0, 20, 500)

# Define reference signal
reference_signal = 50 * np.ones_like(time)

Let us also assume that the slope of the road is $0%$, for the first 10 seconds, and then changes to $10%$ for the next 10 secons:

In [119]:
disturbance = np.zeros_like(time)
disturbance[time >= 10] = 10

### Open-loop, Feed-forward Control

Let us define a few types of control signals so that the output of the system (i.e., car's speed) matches the desired speed (i.e., the reference).

In [120]:
# Define a time-varying input signal (constant)
control_signal_1 = 50 * np.ones_like(time)

# Define a time-varying input signal (ramp for first 5 seconds, then constant)
control_signal_2 = 50 * np.ones_like(time)
control_signal_2[time < 5] = np.linspace(0,50,len(time)//4)

# Define a time-varying input signal (step for first 15 seconds, then constant)
control_signal_3 = 50 * np.ones_like(time)
control_signal_3[time < 15] = np.linspace(0,50,3*len(time)//4)

control_signals = {
    "constant": control_signal_1,
    "ramp_until_5sec": control_signal_2,
    "ramp_until_15sec": control_signal_3
}


This approach to control design is called **open-loop** or **feed-forward** control. Essentially you are relying on your knowledge of the system to try to "reverse engineer" it, and find the control input that generates a desired output. 

Check how good your choice of control signal was by running a simulation:

In [121]:
def simulate_sys1(time, disturbance, control_signal):
    # Define the transfer function
    tf = ct.TransferFunction([0.73], [1, 1])

    # compute disturbed input
    disturbed_input = -2.1 * disturbance + np.clip(control_signal, -100, 100)

    time, response = ct.forced_response(tf, T=time, U=disturbed_input)
    return response

def update_plot(function_name):
    control_signal = control_signals[function_name]

    response = simulate_sys1(time, disturbance, control_signal)

    plt.figure()
    plt.plot(time, control_signal, label="Control Signal", linestyle='-')
    plt.plot(time, response, label="System Response (km/h)")
    plt.plot(time, reference_signal, label="Reference Signal (km/h)", linestyle='--')
    plt.title('1st Order System - Feedforward')
    plt.xlabel('Time (s)')
    plt.ylabel('Speed (km/h)')
    plt.legend()
    plt.show()

# Create a dropdown menu for control signal selection
control_signal_dropdown = widgets.Dropdown(
    options=list(control_signals.keys()),  # Control signal options
    value="constant",              # Default value
    description="Control Signal"
)

# Link the dropdown menu to the update_plot function
widgets.interactive(update_plot, function_name=control_signal_dropdown)

interactive(children=(Dropdown(description='Control Signal', options=('constant', 'ramp_until_5sec', 'ramp_unt…

How good of a job can you do by choosing a control signal this way?

Probably not great, especially since your strategy to come up with a control signal would have to be executed again for each change in the car's performance (e.g., how many passengers are on board), and in the profile of the road.

### Closed-loop,feedback control


The big problem, as you probably realized, is that you were not able to use the information of the behavior of the system in an on-line fashion, but you had to rely on, e.g., a trial and error strategy. 

Clearly, this is not what you do when you actually drive a car (or a bike, or walking, etc...). In these cases, you can perceive your speed based on your senses, and possibly sensors in the vehicle, and then use this information to regulate how much to press or depress the gas pedal. 

What does this new control architecture look like?

![Image Title](./media/sys1fb.png)

How do we get to this block diagram and control architecture? The idea is as follows:
1. We measure the output of interest (in this case, speed) with a sensor;
2. We tap that signal and compare it to the reference, i.e., compute the difference between the desired, reference speed, and the actual, measured speed. This is the speed **error**. If the actual speed is lower than the desired speed, we would like to increase the gas pedal %. Viceversa if the actual speed is higher than the desired speed. 
3. The amount by which we increase the gas for a given speed is called the **control gain**. In this case, the control gain **$K$** measures how much % we increase the gas pedal for a $1$ km/h error in speed. Notice that this is a **negative feedback** because if the speed is lower, the gas command is higher (notice the negative sign in the block combining the measured output and the reference signal). 

In [122]:
def simulate_sys1fb(time, disturbance, K):
    # Define the transfer function
    tf = ct.TransferFunction([0.73], [1, 1])

    # Initialize variables for simulation
    control_signal = np.zeros_like(time)
    response = np.zeros_like(time)
    disturbed_input = np.zeros_like(time)

    # Simulate the system response over time
    for i in range(1, len(time)):
        # Calculate error (reference - current output)
        error = reference_signal[i] - response[i-1]
        
        # Compute control signal u based on the error
        control_signal[i] = np.clip(K * error, -100, 100)

        # Add disturbance and clip the control signal
        disturbed_input[i] = -2.1 * disturbance[i] + control_signal[i]

        # Simulate the system's response to this disturbed input
        t, y = ct.forced_response(tf, T=[time[:i+1]], U=[disturbed_input[:i+1]])

        # Update the system response at this time step
        response[i] = y[-1]
    
    control_signal[0] = control_signal[1] # for visualization purposes
    return response, control_signal

# Initialize a display object
output = widgets.Output()

# Function to update the plot and title
def update_plot(k):
    with output:
        clear_output(wait=True)  # Clear the previous output
        response, control_signal = simulate_sys1fb(time, disturbance, k)

        plt.figure()
        plt.plot(time, control_signal, label="Control Signal", linestyle='-')
        plt.plot(time, response, label="System Response (km/h)")
        plt.plot(time, reference_signal, label="Reference Signal (km/h)", linestyle='--')
        plt.title('1st Order System - Feedback, K = {}'.format(k))
        plt.xlabel('Time (s)')
        plt.legend()
        plt.show()

# Create a slider for b
b_slider = widgets.FloatSlider(value=10, min=-20, max=20, step=1.0, description="k")

# Link the slider to the update_plot function using interactive_output
widgets.interactive(update_plot, k = b_slider)

# Display the slider and the plot
display(b_slider, output)


FloatSlider(value=10.0, description='k', max=20.0, min=-20.0, step=1.0)

Output()

Try to see what happens with different (positive) values of $K$:
- How closely does the output match the reference?
- What is the effect of the disturbance?
- How quickly does the system react?
- How aggressive is your control? How comfortable do you think it would be to be a passenger in that car? What about fuel efficiency?

Now try with negative values of $K$. What happens? Why?

### Two-degrees-of-freedom Control



One limitation of (proportional) feedback control, is that it needs an error of some sort to figure out a non-zero control gain. In other words, the error needs to be large enough to elicit a reaction in the proportional feedback control. 

What if we combined both feed-forward and feedback? See for yourself with the system below:

![Image Title](./media/sys1twodof.png)

In [130]:
def simulate_sys1twodof(time, disturbance, control_signal_designer, K):
    # Define the transfer function
    tf = ct.TransferFunction([0.73], [1, 1])

    # Initialize variables for simulation
    control_signal = np.zeros_like(time)
    response = np.zeros_like(time)
    disturbed_input = np.zeros_like(time)

    # Simulate the system response over time
    for i in range(1, len(time)):
        # Calculate error (reference - current output)
        error = reference_signal[i] - response[i-1]
        
        # Compute control signal u based on the error
        control_signal[i] = np.clip(K * error + control_signal_designer[i], -100, 100)

        # Add disturbance and clip the control signal
        disturbed_input[i] = -2.1 * disturbance[i] + control_signal[i]

        # Simulate the system's response to this disturbed input
        t, y = ct.forced_response(tf, T=[time[:i+1]], U=[disturbed_input[:i+1]])

        # Update the system response at this time step
        response[i] = y[-1]
    control_signal[0] = control_signal[1] # for visualization purposes

    return response, control_signal

# Initialize a display object
output = widgets.Output()

# Function to update the plot and title
def update_plot(k, control_signal_name):
    with output:
        clear_output(wait=True)  # Clear the previous output

        # Get the control signal based on the dropdown selection
        control_signal = control_signals[control_signal_name]
        
        # Simulate the system response (you can replace this with your actual simulation)
        response, control_signal_simulated = simulate_sys1twodof(time, disturbance, control_signal, k)

        # Plot the results
        plt.figure()
        plt.plot(time, control_signal_simulated, label="Control Signal", linestyle='-')
        plt.plot(time, response, label="System Response (km/h)")
        plt.plot(time, reference_signal, label="Reference Signal (km/h)", linestyle='--')
        plt.title('Two-degrees-of-freedom, K = {}'.format(k))
        plt.xlabel('Time (s)')
        plt.legend()
        plt.show()

# Create a slider for k
k_slider = widgets.FloatSlider(value=10, min=-20, max=20, step=1.0, description="k")

# Create a dropdown menu for control signal selection
control_signal_dropdown = widgets.Dropdown(
    options=list(control_signals.keys()),  # Control signal options
    value="constant",              # Default value
    description="Control Signal"
)

# Output widget to display the plot
output = widgets.Output()

# Link the slider and dropdown to the update_plot function using interactive_output
widgets.interactive_output(update_plot, {"k": k_slider, "control_signal_name": control_signal_dropdown})

# Display the widgets and the output
display(k_slider, control_signal_dropdown, output)



FloatSlider(value=10.0, description='k', max=20.0, min=-20.0, step=1.0)

Dropdown(description='Control Signal', options=('constant', 'ramp_until_5sec', 'ramp_until_15sec'), value='con…

Output()

In summary, the two-degrees-of-freedom architecture allows the designer to give a "hint" to the controller, so that it does not need a large error in order to generate an appropriate control signal.

Note that if we set K=0, we recover a purely feed-forward control architecture. Conversely, if we set *control_signal*$=0$, we recover a pure feedback control architecture. Hence, the two-degrees-of-freedom architecture generalizes the other two architectures, and indeed there is nothing wrong in adding a reasonable feedforward in any control design, even though we will focus on feedback control in this course. 

### Second-order system



Consider now the problem of controlling the altitude of the helicopter, using the model:

![Image Title](./media/sys2twodof.png)

Let's assume that we want the helicopter to take off and hover at 50 meters of altitude

In [132]:
def simulate_sys2twodof(time, disturbance, control_signal_designer, K):
    tf = ct.TransferFunction([1], [1, 1, 0])

    # Initialize variables for simulation
    control_signal = np.zeros_like(time)
    response = np.zeros_like(time)
    disturbed_input = np.zeros_like(time)

    # Simulate the system response over time
    for i in range(1, len(time)):
        # Calculate error (reference - current output)
        error = reference_signal[i] - response[i-1]
        
        # Compute control signal u based on the error
        control_signal[i] = np.clip(K * error + control_signal_designer[i], -100, 100)

        # Add disturbance and clip the control signal
        disturbed_input[i] = -1* (50+disturbance[i]) + control_signal[i]

        # Simulate the system's response to this disturbed input
        t, y = ct.forced_response(tf, T=[time[:i+1]], U=[disturbed_input[:i+1]])

        # Update the system response at this time step
        response[i] = y[-1]
    control_signal[0] = control_signal[1] # for visualization purposes

    return response, control_signal

# Initialize a display object
output = widgets.Output()

# Function to update the plot and title
def update_plot(k, control_signal_name):
    with output:
        clear_output(wait=True)  # Clear the previous output

        # Get the control signal based on the dropdown selection
        control_signal = control_signals[control_signal_name]
        
        # Simulate the system response (you can replace this with your actual simulation)
        response, control_signal_simulated = simulate_sys2twodof(time, disturbance, control_signal, k)

        # Plot the results
        plt.figure()
        plt.plot(time, control_signal_simulated, label="Control Signal", linestyle='-')
        plt.plot(time, response, label="System Response (km/h)")
        plt.plot(time, reference_signal, label="Reference Signal (km/h)", linestyle='--')
        plt.title('2nd Order System - K = {}'.format(k))
        plt.xlabel('Time (s)')
        plt.legend()
        plt.show()

# Create a slider for k
k_slider = widgets.FloatSlider(value=1, min=-20, max=20, step=1.0, description="k")

# Create a dropdown menu for control signal selection
control_signal_dropdown = widgets.Dropdown(
    options=list(control_signals.keys()),  # Control signal options
    value="constant",              # Default value
    description="Control Signal"
)

# Output widget to display the plot
output = widgets.Output()

# Link the slider and dropdown to the update_plot function using interactive_output
widgets.interactive_output(update_plot, {"k": k_slider, "control_signal_name": control_signal_dropdown})

# Display the widgets and the output
display(k_slider, control_signal_dropdown, output)

FloatSlider(value=1.0, description='k', max=20.0, min=-20.0, step=1.0)

Dropdown(description='Control Signal', options=('constant', 'ramp_until_5sec', 'ramp_until_15sec'), value='con…

Output()

Try different values of K, see how close to the target altitude the helicopter will get. What happens if you make the gain K "too large"?

### Higher-order system

Consider the following higher-order system:

![Image Title](./media/sys3fb.png)

In [133]:
def simulate_sys3fb(time, disturbance, K):
    tf = ct.TransferFunction([1], [1, 1, 0, 0])

    # Initialize variables for simulation
    control_signal = np.zeros_like(time)
    response = np.zeros_like(time)
    disturbed_input = np.zeros_like(time)

    # Simulate the system response over time
    for i in range(1, len(time)):
        # Calculate error (reference - current output)
        error = reference_signal[i] - response[i-1]
        
        # Compute control signal u based on the error
        control_signal[i] = np.clip(K * error, -100, 100)

        # Add disturbance and clip the control signal
        disturbed_input[i] = -1 * disturbance[i] + control_signal[i]

        # Simulate the system's response to this disturbed input
        t, y = ct.forced_response(tf, T=[time[:i+1]], U=[disturbed_input[:i+1]])

        # Update the system response at this time step
        response[i] = y[-1]
    
    control_signal[0] = control_signal[1] # for visualization purposes
    return response, control_signal

# Initialize a display object
output = widgets.Output()

# Function to update the plot and title
def update_plot(k):
    with output:
        clear_output(wait=True)  # Clear the previous output
        response, control_signal = simulate_sys3fb(time, disturbance, k)

        plt.figure()
        plt.plot(time, control_signal, label="Control Signal", linestyle='-')
        plt.plot(time, response, label="System Response (km/h)")
        plt.plot(time, reference_signal, label="Reference Signal (km/h)", linestyle='--')
        plt.title('Higher Order System - K = {:.1f}'.format(k))
        plt.xlabel('Time (s)')
        plt.xlim([0, 20])
        plt.ylim([-100,100])
        plt.legend()
        plt.show()

# Create a slider for b
b_slider = widgets.FloatSlider(value=0.1, min=-20, max=20, step=0.1, description="k")

# Link the slider to the update_plot function using interactive_output
widgets.interactive(update_plot, k = b_slider)

# Display the slider and the plot
display(b_slider, output)

FloatSlider(value=0.1, description='k', max=20.0, min=-20.0)

Output()

What happens as you change K? Can you find "good" values of the control gain?

Why do you think the system behavior under feedback is so different in this case?

We will answer all these questions, and more, in the following weeks. 